<h1><center>2. Unstructured Data Classification</center></h1>

<p style='text-align: center;'> 
Jishnu Jeevan <br>
Department of Computer Science <br>
M.Tech Computer and Information Science <br>
jishnujeevan@cusat.ac.in <br>
</p>

<h2><center> Assignemt Objective</center></h2>
<p style='text-align: justify;'>
These 30 scientific articles come from three different domains:<br>
PLoS Computational Biology (PLOS)<br>
The machine learning repository on arXiv (ARXIV)<br>
The psychology journal Judgment and Decision Making (JDM)<br>
There are 10 articles from each domain. In addition to the labeled data, this corpus also contains a corresponding set of unlabeled articles. These unlabeled articles also come from PLOS, ARXIV, and JDM. There are 300 unlabeled articles from each domain (again, only the sentences from the abstract and introduction).<br>
Our objective is to label each sentence in the unlabelled data set with AIMX, OWNX, CONT, BASE, MISC.<br>
Here the training set is smaller than the test set.<br>
</p>

<h2>The data set contains the following information: </h2>
<p style='text-align: justify;'>
    
1. labeled_articles<br>
It has a total of 30 texts (10 text from each domain PLOS, ARXIV, JDM)<br>
Each of the sentence is marked with the following label
Aim - AIMX - The specific research goal of the paper<br>
Own - OWNX - The author’s own work, e.g. methods, results, conclusions<br>
Contrast - CONT - Contrast, comparison or critique of past work<br>
Basis - BASE - Past work that provides the basis for the work in the article.<br>
Misc - MISC - Any other sentences<br>
    
2. unlabelled_articles<br>
They contain three folder from arxiv_unlabelled, plos_unlabelled, jdm_unlabelled. <br>
Each folder contains 300 test files. But these test files don't have there sentence labelled.

3. word_list <br>
Conaints test files named aim, base, contrast, own, stopwords. Each test file contains the words that are commonly used with the labels aim, base, contrast, own. <br>
Stopwords contains the words that don't have any meaning like of, a, and, the etc.
</p>



### 1. Import the required libraries

In [115]:
# For path manipulation
import os

# For showing a progress bar
from tqdm import tqdm

# For regular expression
import re

# For training, testing and splitting of the data
from sklearn.model_selection import train_test_split

# For count vectorizer
from sklearn.feature_extraction.text import CountVectorizer

# For calculating accuracy
from sklearn import metrics, preprocessing

#Classification Algorithms 
from sklearn.model_selection import cross_val_score # Cross validation
from sklearn.tree import DecisionTreeClassifier # Decision Tree
from sklearn.svm import SVC # C - Suport Vector Classification
from sklearn.multiclass import OneVsRestClassifier # One vs rest classifier
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent
from sklearn.neighbors import KNeighborsClassifier # K Neighbours
from sklearn.ensemble import RandomForestClassifier # Random Forrest
from sklearn.naive_bayes import MultinomialNB # Multinominal Naive Bayes

### 2. Read the file and seperate the labels and the sentence. 
### Here a sentence will be of the form: AIMX"tab"We show that this is even the case if the model class contains only Bernoulli distributions. 
### The first word AIMX is the label and the rest that follows after the tab is the sentence. I coudn't display tab, so I wrote it as "tab"
### Due to the fact that many text documents were not seperated by tab, I had to manually insert tab after the labels.

In [116]:
# Path of the training data
Training_Folder = './SentenceCorpus/labeled_articles'

# Labels
Training_Label = []

# Sentence
Training_Matrix = []

# Take each file from labeled_articles
for file in tqdm(os.listdir(Training_Folder)):
        
    # Open each file in reaf mode
    Text_File = open(Training_Folder + "/" + file,'r+')
    
    # Take each line of the file
    for line in Text_File:

        # If it starts with ### dont process it
        if (line.startswith('###')):
            continue

        # Reomve other characters like , . | etc with a space 
        line = re.sub(r'[-*+%$()\.,/?!><"&#\[\]\(\)\\]', ' ',line)
        
        # Split the sentace by tab
        lines = line.split("\t")
        #print(lines)
        
        # Put the sentence into list and convert it to lower case
        Training_Matrix.append(lines[1].lower())

        # Put the labels in list
        Training_Label.append(lines[0])
    Text_File.close()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 1500.44it/s]


### 3. Now we are going to do pre processing on the words before we feed them to a training mode

#### 3a. Tokenization - Converting sentence to words

In [117]:
# import the nltk (natural language took kit)
import nltk

# Import word tokenizer
from nltk.tokenize import word_tokenize

# Create a list to store the tokenized form of each sentence
Training_Matrix_tokenized = []

# Take every sentence in training matrix
for sentence in Training_Matrix:
    
    # Append the tokenized form of the sentence
    Training_Matrix_tokenized.append(word_tokenize(sentence))

print(Training_Matrix[0])
print(Training_Matrix_tokenized[0])

the minimum description length principle for online sequence estimation prediction in a proper learning setup is studied

['the', 'minimum', 'description', 'length', 'principle', 'for', 'online', 'sequence', 'estimation', 'prediction', 'in', 'a', 'proper', 'learning', 'setup', 'is', 'studied']


#### 3b. Stop word removal - Frequent words such as ”the”, ”is”, etc. that do not have specific semantic. 
#### We already have a file which contains all the stop words. We are going to use the words in that file to remove the stop words.

In [118]:
# This is the path of the stop word file
Stop_Word_Folder = './SentenceCorpus/word_lists/stopwords.txt'

# Create a list to store all the stop words
stop_words = []

# Open the stop word file
Stop_Word_File = open(Stop_Word_Folder,'r+')

# Take each word of the stop word file
for word in Stop_Word_File:
    
    # When we take the word from the file, a new line will also be read. Remove that new line
    word = word.split('\n')
    
    # Append only the word, and not the new line to the list of stop words
    stop_words.append(word[0])
    
# Close the file
Stop_Word_File.close()
print(stop_words)

['of', 'a', 'and', 'the', 'in', 'to', 'for', 'that', 'is', 'on', 'are', 'with', 'as', 'by', 'be', 'an', 'which', 'it', 'from', 'or', 'can', 'have', 'these', 'has', 'such']


#### Now remove all the stop words from the sentence

In [119]:
# Create a list to store all the words that are not tokens
Training_Matrix_Stop_Words_Removed = []

# Take each row of the training matrix tokenized
for sentence in Training_Matrix_tokenized:
    
    # Create a list to store all the words that are not stop words
    tokens = []
    
    # Take each word of the sentence in the row of the training matrix tokenized
    for word in sentence:
        
        # If the  word is not not a stop word
        if word not in stop_words:
            
            # Append it to the token list
            tokens.append(word)
    
    # Append the stop word removed row to the new training matrix
    Training_Matrix_Stop_Words_Removed.append(tokens)

print(Training_Matrix_Stop_Words_Removed[0])

['minimum', 'description', 'length', 'principle', 'online', 'sequence', 'estimation', 'prediction', 'proper', 'learning', 'setup', 'studied']


#### 3c. Stemming — words are reduced to a root by removing inflection through dropping unnecessary characters, usually a suffix.
#### The stemmed form of studies is: studi
#### The stemmed form of studying is: study

In [120]:
# Import PorterStemmer from nltk
from nltk.stem.porter import PorterStemmer

# Initialize porter stemmer
porter = PorterStemmer()

# Create a list to store all the stemmed words
Training_Matrix_Stemmed = []

# Take each row of the training matrix, that had the stop words removed
for sentence in Training_Matrix_Stop_Words_Removed:
    
    # Create a list to store the stemmed version of each word
    stems = []
    
    # Take each word of the sentence, in the row of the training matrix stop word removed
    for word in sentence:
        
        # Append the stemmed version of the word to the stem list
        stems.append(porter.stem(word))
    
    # Append the stemmed word list to new training matrix
    Training_Matrix_Stemmed.append(stems)

print(Training_Matrix_Stemmed[0])

['minimum', 'descript', 'length', 'principl', 'onlin', 'sequenc', 'estim', 'predict', 'proper', 'learn', 'setup', 'studi']


#### 3d - Count vectorizer - Convert a collection of text documents to a matrix of token counts. 

#### Create a data frame

In [121]:
# For creating a data frame
import pandas as pd

# create a dataframe using texts and lables
trainDF = pd.DataFrame()

# If we want it properly work, the Training_Matrix_Stemmed contain the following:
# ['minimum','descript','length','principl','onlin','sequenc','estim','predict','proper','learn','setup','studi']
# This is a sentence that has been tokenized, removed of stop words and stemmed
# For proper processing as a data frame, we need to convert the above list of words into a sentence like format
# i.e. 'minimum descript length principl onlin sequenc estim predict proper learn setup studi'

# To store all the new sentences
training_data = []

# Take each row of the Training_Matrix_Stemmed:
for sentence in Training_Matrix_Stemmed:
    
    # Join using space
    new_sentence = " "
    
    # Join each word with a space to form a sentence
    new_sentence = new_sentence.join(sentence) 
    
    # Append it to new training matrix
    training_data.append(new_sentence)

trainDF['text'] = training_data
trainDF['label'] = Training_Label

#### Split the data set 

In [122]:
# split the dataset into training and validation datasets 
X_train, X_test, y_train, y_test = train_test_split(trainDF['text'], trainDF['label'])

#### Do count vectorization on training data

In [123]:
from sklearn.feature_extraction.text import CountVectorizer

# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
X_train_count =  count_vect.transform(X_train)
X_test_count =  count_vect.transform(X_test)

### 4. We are going to use different binary classification algorithms to do classification on the data, and we will evaluate the accuarcy of each algorithm.
### The algorithms we are going to use are
1. Decision Tree Classifier
2. Stochastic gradient descent  Classifier
3. SVC
4. K Nearest Neighbour
5. One vs Rest Classifier
6. Random Forrest Classifier
7. Multinominal Naive Bayes

In [124]:
# To surpress warning messages
import warnings
warnings.filterwarnings('ignore')

# Create a dictionary to find out the best classifier using accuracy score
ranking = {}

# We are going to be using the following classifiers and doing a comparision study
classifiers = {
                '1. Decision Tree Classifier':DecisionTreeClassifier(),
                '2. Stochastic Gradient Descent':SGDClassifier(max_iter=1000, tol=1e-3), 
                '3. C - Support Vector Classifier':SVC(gamma='auto'), 
                '4. K Nearest Neighbour':KNeighborsClassifier(8),   
                '5. One vs Rest Classifier': OneVsRestClassifier(SVC(kernel='linear')), 
                '6. Random Forest Classifier': RandomForestClassifier(),
                '7. Multinominal Naive Bayes': MultinomialNB()
               }

# Take each classifier from list
for Name, classifier in classifiers.items():
    # Fit the model using the training set
    classifier.fit(X_train_count ,y_train)
    
    # Find out the predicion using test set
    y_predicted = classifier.predict(X_test_count)
    
    # Find out the accuracy using the y test set and perdicted valur of y
    accuracy = metrics.accuracy_score(y_test,y_predicted)
    
    # Cross validaion score
    score = cross_val_score(classifier, X_train_count, y_train, cv=3)
    
    # Print results
    print("\n")
    print("Name : ", Name)
    print("Accuracy : ", accuracy)
    print("Cross validation score : ", score)
       
    # Add the name of classifier and accuracy score to dictionary
    ranking[Name] = accuracy



Name :  1. Decision Tree Classifier
Accuracy :  0.3333333333333333
Cross validation score :  [0.3307888  0.36970475 0.35233161]


Name :  2. Stochastic Gradient Descent
Accuracy :  0.3474358974358974
Cross validation score :  [0.32315522 0.37355584 0.38341969]


Name :  3. C - Support Vector Classifier
Accuracy :  0.2935897435897436
Cross validation score :  [0.30916031 0.31065469 0.3134715 ]


Name :  4. K Nearest Neighbour
Accuracy :  0.3628205128205128
Cross validation score :  [0.31806616 0.36713736 0.32901554]


Name :  5. One vs Rest Classifier
Accuracy :  0.37435897435897436
Cross validation score :  [0.34732824 0.40693196 0.38212435]


Name :  6. Random Forest Classifier
Accuracy :  0.3243589743589744
Cross validation score :  [0.34860051 0.37098845 0.34715026]


Name :  7. Multinominal Naive Bayes
Accuracy :  0.4166666666666667
Cross validation score :  [0.37913486 0.40051348 0.38341969]


### 5. The aglorithms that perform well, accorging to there accuracy score are as follows:

In [125]:
# Sort the dictionary 'ranking' accoriding to highest accuracy
print("\n")
ranking_sorted = sorted(ranking.items(),  reverse = True, key=lambda x: x[1]) # This returns a tuple, not a dictinary
for k,v in ranking_sorted:
    print(k, ";", v)



7. Multinominal Naive Bayes ; 0.4166666666666667
5. One vs Rest Classifier ; 0.37435897435897436
4. K Nearest Neighbour ; 0.3628205128205128
2. Stochastic Gradient Descent ; 0.3474358974358974
1. Decision Tree Classifier ; 0.3333333333333333
6. Random Forest Classifier ; 0.3243589743589744
3. C - Support Vector Classifier ; 0.2935897435897436


### 6. Now from the accuracy score we can see that Multi nominal naive bayes perfomed well. Now we can use this to do predictions on unlabelled data set

In [127]:
# Create a multi nominal naive bayes
model =  MultinomialNB()

# Train the model again
model.fit(X_train_count ,y_train)

# Specify the path of the test data
path = './SentenceCorpus/unlabeled_articles/arxiv_unlabeled/1.txt'

# Create a variable to store the sentence
X_test = []

# Open each file in reaf mode
Text_File = open(path,'r+')
    
# Take each line of the file
for line in Text_File:

    # If it starts with ### dont process it
    if (line.startswith('###')):
        continue

    # Reomve other characters like , . | etc with a space 
    line = re.sub(r'[-*+%$()\.,/?!><"&#\[\]\(\)\\]', ' ',line)

    # Put the sentence into list and convert it to lower case
    X_test.append(line.lower())

# Close the file
Text_File.close()

# Convert it to a pandas data frame
import pandas as pd
testDF = pd.DataFrame()
testDF['text'] = X_test

# Convert it to count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])
X_test_count =  count_vect.transform(X_test)

# Make prediction
y_predicted = model.predict(X_test_count)

# Print the prediction
print(y_predicted)

['MISC' 'MISC ' 'AIMX' 'MISC' 'MISC' 'OWNX ' 'MISC ' 'MISC ' 'MISC '
 'MISC ' 'OWNX' 'MISC ' 'MISC ' 'OWNX' 'MISC' 'MISC ' 'MISC ' 'MISC '
 'MISC ' 'MISC ' 'AIMX ' 'MISC' 'MISC ' 'MISC' 'MISC ' 'MISC' 'MISC'
 'MISC ' 'AIMX' 'OWNX' 'MISC ' 'MISC ' 'MISC ']
